In [1]:
import sys
print(sys.version)

3.10.6 (main, Nov  2 2022, 18:53:38) [GCC 11.3.0]


In [2]:
import sklearn.ensemble
import shap
import numpy as np
import numba
import time
import xgboost

/home/jarek/projects/shap/venv/lib/python3.10/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
X,y = shap.datasets.boston()
X.shape



Function load_boston is deprecated; `load_boston` is deprecated in 1.0 and will be removed in 1.2.

    The Boston housing prices dataset has an ethical problem. You can refer to
    the documentation of this function for further details.

    The scikit-learn maintainers therefore strongly discourage the use of this
    dataset unless the purpose of the code is to study and educate about
    ethical issues in data science and machine learning.

    In this special case, you can fetch the dataset from the original
    source::

        import pandas as pd
        import numpy as np

        data_url = "http://lib.stat.cmu.edu/datasets/boston"
        raw_df = pd.read_csv(data_url, sep="\s+", skiprows=22, header=None)
        data = np.hstack([raw_df.values[::2, :], raw_df.values[1::2, :2]])
        target = raw_df.values[1::2, 2]

    Alternative datasets include the California housing dataset (i.e.
    :func:`~sklearn.datasets.fetch_california_housing`) and the Ames housing
    datase

(506, 13)

In [4]:
model = sklearn.ensemble.RandomForestRegressor(n_estimators=100, max_depth=6)
model.fit(X, y)



RandomForestRegressor(max_depth=6)

In [5]:
bst = xgboost.train({"learning_rate": 0.01, "max_depth": 4}, xgboost.DMatrix(X, label=y), 1000)

In [6]:
start = time.time()
shap_values = bst.predict(xgboost.DMatrix(X), pred_contribs=True)
print(time.time() - start)


0.0751495361328125


In [7]:
from shap.explainers.pytree import *
# x = np.ones(X.shape[1])
# TreeExplainer(model).shap_values(x)

In [8]:
from shap import TreeExplainer as TreeCext
start = time.time()
t = TreeCext(model).shap_values(X, banz=False)
print(time.time() - start)
t

ile drzew cext? 100
using treeshap and cext
banz = False
here37
using dense_Tree_pred
0.18568110466003418


array([[ 2.77456839e-01, -9.59061940e-03,  9.08763791e-02, ...,
         2.73324646e-01,  9.42110610e-03,  4.54826403e+00],
       [ 3.58244526e-01, -5.01015989e-03, -2.08460771e-02, ...,
         1.72229676e-01,  2.14324361e-02,  1.84252282e+00],
       [ 2.93006083e-01, -1.22709199e-02,  9.45214588e-03, ...,
         1.32981466e-01,  7.20081760e-02,  5.04567220e+00],
       ...,
       [ 3.61047012e-01, -1.00187336e-01, -3.92389770e-01, ...,
        -1.11246016e+00, -9.89655937e-02,  2.73892991e+00],
       [ 3.15310701e-01, -1.26665356e-02, -6.65899792e-02, ...,
        -9.38786083e-01,  6.42598983e-02,  3.62050105e+00],
       [ 3.39625367e-01, -7.94136325e-03, -4.80361585e-02, ...,
        -5.65761906e-01,  3.26243915e-02,  1.56168175e+00]])

In [9]:
start = time.time()
t = TreeCext(model).shap_values(X, banz=True)
print(time.time() - start)
t # TODO BLAD - dodają się zamiast zaczynac od 0 dla kazdej probki danych

ile drzew cext? 100
using treeshap and cext
banz = True
using cext banz
0.23976802825927734


array([[ 2.79732606e-01, -8.96269728e-03,  9.68243893e-02, ...,
         2.92334355e-01,  1.89463516e-02,  4.59603504e+00],
       [ 5.59465213e-01, -1.79253946e-02,  1.93648779e-01, ...,
         5.84668709e-01,  3.78927032e-02,  9.19207007e+00],
       [ 8.39197819e-01, -2.68880918e-02,  2.90473168e-01, ...,
         8.77003064e-01,  5.68390548e-02,  1.37881051e+01],
       ...,
       [ 1.40985234e+02, -4.51719943e+00,  4.87994922e+01, ...,
         1.47336515e+02,  9.54896120e+00,  2.31640166e+03],
       [ 1.41264966e+02, -4.52616212e+00,  4.88963166e+01, ...,
         1.47628849e+02,  9.56790756e+00,  2.32099769e+03],
       [ 1.41544699e+02, -4.53512482e+00,  4.89931410e+01, ...,
         1.47921183e+02,  9.58685391e+00,  2.32559373e+03]])

In [10]:
start = time.time()
ex = TreeExplainer(model)
print(time.time() - start)
start = time.time()
print(ex.shap_values(X))
print(time.time() - start)

0.03676891326904297
oryginalna implemenacja shap_values
other model type
using slow python treeshap
[[ 2.77456839e-01 -9.59061940e-03  9.08763791e-02 ...  9.42110610e-03
   4.54826403e+00  2.24464229e+01]
 [ 3.58244526e-01 -5.01015989e-03 -2.08460771e-02 ...  2.14324361e-02
   1.84368822e+00  2.24464229e+01]
 [ 2.93006083e-01 -1.22709199e-02  9.45214588e-03 ...  7.20081760e-02
   5.04683761e+00  2.24464229e+01]
 ...
 [ 3.62278886e-01 -1.00187336e-01 -3.92389770e-01 ... -9.89655937e-02
   2.73892991e+00  2.24464229e+01]
 [ 3.15354710e-01 -1.26665356e-02 -6.65899792e-02 ...  6.42598983e-02
   3.62050105e+00  2.24464229e+01]
 [ 3.39669376e-01 -7.94136325e-03 -4.80361585e-02 ...  3.26243915e-02
   1.56168175e+00  2.24464229e+01]]
95.35734939575195


In [11]:
start = time.time()
res = ex.banz_values(X)
print(time.time() - start)
res

nasza implementacja banz values
other model type
uzywajac pythonowego banzhafa
using numba py banz
dwa wymiary
121.97282934188843


[[0.2797326062684291,
  -0.008962697276228528,
  0.09682438932328619,
  -0.0023495908643153835,
  0.0695021780866027,
  -0.9088840265478,
  -0.010258521366639926,
  -0.2263986931345072,
  -0.10659953528204605,
  -0.13098091478248683,
  0.29233435460614543,
  0.01894635159642633,
  4.596035035015582],
 [0.3655999996613879,
  -0.005309498270322884,
  -0.022529187536586434,
  -0.002933102310504226,
  0.401610889874278,
  -2.310020022433462,
  -0.0044579678904762285,
  -0.2341493416971242,
  -0.01942539745100067,
  0.020257268176108737,
  0.1778866773112746,
  0.03210791292568799,
  1.8513170544869455],
 [0.2576419501234495,
  -0.01435047992294509,
  -0.0004685172004792171,
  -0.011652460462608476,
  0.4100450638680348,
  5.9760555786518355,
  0.055061561269079524,
  -0.25975717430094936,
  -0.07968971706444745,
  0.24829999472325667,
  0.12523145753447262,
  0.06627558027315407,
  4.933134440232745],
 [0.2768517164989049,
  -0.057554288954110466,
  0.13011169253290536,
  -0.01441520045760

In [12]:
start = time.time()
res = ex.banz_values(X, jit=True)
print(time.time() - start)
res

nasza implementacja banz values
other model type
uzywajac pythonowego banzhafa
dwa wymiary
121.70343947410583


[[0.2797326062684291,
  -0.008962697276228528,
  0.09682438932328619,
  -0.0023495908643153835,
  0.0695021780866027,
  -0.9088840265478,
  -0.010258521366639926,
  -0.2263986931345072,
  -0.10659953528204605,
  -0.13098091478248683,
  0.29233435460614543,
  0.01894635159642633,
  4.596035035015582],
 [0.3655999996613879,
  -0.005309498270322884,
  -0.022529187536586434,
  -0.002933102310504226,
  0.401610889874278,
  -2.310020022433462,
  -0.0044579678904762285,
  -0.2341493416971242,
  -0.01942539745100067,
  0.020257268176108737,
  0.1778866773112746,
  0.03210791292568799,
  1.8513170544869455],
 [0.2576419501234495,
  -0.01435047992294509,
  -0.0004685172004792171,
  -0.011652460462608476,
  0.4100450638680348,
  5.9760555786518355,
  0.055061561269079524,
  -0.25975717430094936,
  -0.07968971706444745,
  0.24829999472325667,
  0.12523145753447262,
  0.06627558027315407,
  4.933134440232745],
 [0.2768517164989049,
  -0.057554288954110466,
  0.13011169253290536,
  -0.01441520045760

###### 